In [84]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import streamlit as st
from nltk.corpus import stopwords


In [85]:
df = pd.read_csv('../data/data.csv')
df.head()

,id,listing_url,name,description,picture_url,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,...,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,baths,property_category
0,783493769216852616,https://www.airbnb.com/rooms/783493769216852616,"Leme, Brasil",Apartamento aconchegante próximo à praia do Le...,https://a0.muscache.com/pictures/fe1f4b1b-a300...,100.0,67.0,0,2.0,2.0,...,NaN,NaN,0,2,2,0,0,NaN,1.0,Entire
1,703973293620197060,https://www.airbnb.com/rooms/703973293620197060,Suíte com entrada independente em casarão 1,"Casa ampla, em excelente localização, situada ...",https://a0.muscache.com/pictures/miso/Hosting-...,70.0,100.0,0,4.0,5.0,...,4.00,5.00,1,4,0,4,0,0.26,1.0,Private
2,782895997622988215,https://www.airbnb.com/rooms/782895997622988215,Apartamento próximo ao metrô,Sua família vai estar perto de tudo ao ficar n...,https://a0.muscache.com/pictures/d48a6798-d1c8...,100.0,50.0,0,1.0,1.0,...,NaN,NaN,0,1,1,0,0,NaN,1.0,Entire
3,23768085,https://www.airbnb.com/rooms/23768085,Vamos a praia,"Ótimo apartamento para duas pessoas, de frente...",https://a0.muscache.com/pictures/e43b686c-c4b4...,100.0,100.0,0,1.0,2.0,...,4.98,4.78,1,1,1,0,0,1.59,1.0,Entire
4,10463735,https://www.airbnb.com/rooms/10463735,Sobrado aconchegante e espaçoso,"Sobrado amplo, arejado, com arquitetura inusit...",https://a0.muscache.com/pictures/52ffaa78-2e94...,78.0,100.0,0,6.0,6.0,...,2.00,1.00,1,6,2,4,0,0.03,2.0,Entire


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28291 entries, 0 to 28290
Data columns (total 50 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            28291 non-null  int64  
 1   listing_url                                   28291 non-null  object 
 2   name                                          28262 non-null  object 
 3   description                                   27532 non-null  object 
 4   picture_url                                   28291 non-null  object 
 5   host_response_rate                            23888 non-null  float64
 6   host_acceptance_rate                          24542 non-null  float64
 7   host_is_superhost                             28291 non-null  int64  
 8   host_listings_count                           28291 non-null  float64
 9   host_total_listings_count                     28291 non-null 

In [87]:
df.columns

Index(['id', 'listing_url', 'name', 'description', 'picture_url',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_listings_count', 'host_total_listings_count',
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'accommodates',
       'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights',
       'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'number_of_reviews_ltm',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable',
       'calculated_

In [88]:
df.isnull().sum()

id                                                 0
listing_url                                        0
name                                              29
description                                      759
picture_url                                        0
host_response_rate                              4403
host_acceptance_rate                            3749
host_is_superhost                                  0
host_listings_count                                0
host_total_listings_count                          0
host_has_profile_pic                               0
host_identity_verified                             0
neighbourhood_cleansed                             0
latitude                                           0
longitude                                          0
accommodates                                       0
bedrooms                                        1281
beds                                             216
amenities                                     

In [89]:
col = ['minimum_nights', 'minimum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'instant_bookable']

df.drop(col, axis=1, inplace=True)
df.head()


,id,listing_url,name,description,picture_url,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,...,number_of_reviews_ltm,review_scores_rating,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,baths,property_category
0,783493769216852616,https://www.airbnb.com/rooms/783493769216852616,"Leme, Brasil",Apartamento aconchegante próximo à praia do Le...,https://a0.muscache.com/pictures/fe1f4b1b-a300...,100.0,67.0,0,2.0,2.0,...,0,NaN,NaN,2,2,0,0,NaN,1.0,Entire
1,703973293620197060,https://www.airbnb.com/rooms/703973293620197060,Suíte com entrada independente em casarão 1,"Casa ampla, em excelente localização, situada ...",https://a0.muscache.com/pictures/miso/Hosting-...,70.0,100.0,0,4.0,5.0,...,1,5.00,5.00,4,0,4,0,0.26,1.0,Private
2,782895997622988215,https://www.airbnb.com/rooms/782895997622988215,Apartamento próximo ao metrô,Sua família vai estar perto de tudo ao ficar n...,https://a0.muscache.com/pictures/d48a6798-d1c8...,100.0,50.0,0,1.0,1.0,...,0,NaN,NaN,1,1,0,0,NaN,1.0,Entire
3,23768085,https://www.airbnb.com/rooms/23768085,Vamos a praia,"Ótimo apartamento para duas pessoas, de frente...",https://a0.muscache.com/pictures/e43b686c-c4b4...,100.0,100.0,0,1.0,2.0,...,24,4.79,4.78,1,1,0,0,1.59,1.0,Entire
4,10463735,https://www.airbnb.com/rooms/10463735,Sobrado aconchegante e espaçoso,"Sobrado amplo, arejado, com arquitetura inusit...",https://a0.muscache.com/pictures/52ffaa78-2e94...,78.0,100.0,0,6.0,6.0,...,0,3.00,1.00,6,2,4,0,0.03,2.0,Entire


In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28291 entries, 0 to 28290
Data columns (total 38 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            28291 non-null  int64  
 1   listing_url                                   28291 non-null  object 
 2   name                                          28262 non-null  object 
 3   description                                   27532 non-null  object 
 4   picture_url                                   28291 non-null  object 
 5   host_response_rate                            23888 non-null  float64
 6   host_acceptance_rate                          24542 non-null  float64
 7   host_is_superhost                             28291 non-null  int64  
 8   host_listings_count                           28291 non-null  float64
 9   host_total_listings_count                     28291 non-null 

In [91]:
df.property_category

0         Entire
1        Private
2         Entire
3         Entire
4         Entire
          ...   
28286     Entire
28287     Entire
28288     Entire
28289     Entire
28290    Private
Name: property_category, Length: 28291, dtype: object

In [92]:
df.isnull().sum()

id                                                 0
listing_url                                        0
name                                              29
description                                      759
picture_url                                        0
host_response_rate                              4403
host_acceptance_rate                            3749
host_is_superhost                                  0
host_listings_count                                0
host_total_listings_count                          0
host_has_profile_pic                               0
host_identity_verified                             0
neighbourhood_cleansed                             0
latitude                                           0
longitude                                          0
accommodates                                       0
bedrooms                                        1281
beds                                             216
amenities                                     

In [93]:
# def replace_missing():
    # replace missing values of columns with text 'missing'
df['name'].fillna('missing', inplace=True)
df['description'].fillna('missing', inplace=True)

     # replace missing values with median
df['host_response_rate'].fillna(df['host_response_rate'].median(), inplace=True)    
df['host_acceptance_rate'].fillna(df['host_acceptance_rate'].median(), inplace=True)
df['review_scores_rating'].fillna(df['review_scores_rating'].median(), inplace=True)
df['review_scores_value'].fillna(df['review_scores_value'].median(), inplace=True)
df['reviews_per_month'].fillna(df['reviews_per_month'].median(), inplace=True)
            
    # replace missing values column with mode
df['bedrooms'].fillna(df['bedrooms'].mode(), inplace=True)
df['beds'].fillna(df['beds'].mode(), inplace=True)

    #return df

In [94]:
df.isnull().sum()

id                                                 0
listing_url                                        0
name                                               0
description                                        0
picture_url                                        0
host_response_rate                                 0
host_acceptance_rate                               0
host_is_superhost                                  0
host_listings_count                                0
host_total_listings_count                          0
host_has_profile_pic                               0
host_identity_verified                             0
neighbourhood_cleansed                             0
latitude                                           0
longitude                                          0
accommodates                                       0
bedrooms                                        1281
beds                                             216
amenities                                     

In [95]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.isnull().sum()

id                                              0
listing_url                                     0
name                                            0
description                                     0
picture_url                                     0
host_response_rate                              0
host_acceptance_rate                            0
host_is_superhost                               0
host_listings_count                             0
host_total_listings_count                       0
host_has_profile_pic                            0
host_identity_verified                          0
neighbourhood_cleansed                          0
latitude                                        0
longitude                                       0
accommodates                                    0
bedrooms                                        0
beds                                            0
amenities                                       0
price                                           0


In [99]:
df.to_csv('../data/df.csv', index=False)


In [112]:
df2 = pd.read_csv('../data/df.csv')
df2.head()

,id,listing_url,name,description,picture_url,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,...,number_of_reviews_ltm,review_scores_rating,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,baths,property_category
0,783493769216852616,https://www.airbnb.com/rooms/783493769216852616,"Leme, Brasil",Apartamento aconchegante próximo à praia do Le...,https://a0.muscache.com/pictures/fe1f4b1b-a300...,100.0,67.0,0,2.0,2.0,...,0,4.91,4.80,2,2,0,0,0.54,1.0,Entire
1,703973293620197060,https://www.airbnb.com/rooms/703973293620197060,Suíte com entrada independente em casarão 1,"Casa ampla, em excelente localização, situada ...",https://a0.muscache.com/pictures/miso/Hosting-...,70.0,100.0,0,4.0,5.0,...,1,5.00,5.00,4,0,4,0,0.26,1.0,Private
2,782895997622988215,https://www.airbnb.com/rooms/782895997622988215,Apartamento próximo ao metrô,Sua família vai estar perto de tudo ao ficar n...,https://a0.muscache.com/pictures/d48a6798-d1c8...,100.0,50.0,0,1.0,1.0,...,0,4.91,4.80,1,1,0,0,0.54,1.0,Entire
3,23768085,https://www.airbnb.com/rooms/23768085,Vamos a praia,"Ótimo apartamento para duas pessoas, de frente...",https://a0.muscache.com/pictures/e43b686c-c4b4...,100.0,100.0,0,1.0,2.0,...,24,4.79,4.78,1,1,0,0,1.59,1.0,Entire
4,10463735,https://www.airbnb.com/rooms/10463735,Sobrado aconchegante e espaçoso,"Sobrado amplo, arejado, com arquitetura inusit...",https://a0.muscache.com/pictures/52ffaa78-2e94...,78.0,100.0,0,6.0,6.0,...,0,3.00,1.00,6,2,4,0,0.03,2.0,Entire


In [104]:
df2.to_csv('../data/df2.csv', index=False)


In [101]:
stop_words_en = stopwords.words('english')
stop_words_po = stopwords.words('portuguese')



In [102]:
features = df2[['name', 'neighbourhood_cleansed', 'description', 'property_category', 'amenities']]

# create the tf-idf matrix
#@st.cache
def create_tfidf_matrix(df2):
    stop_words = stop_words_en + stop_words_po
    tfidf = TfidfVectorizer(stop_words=stop_words)
    tfidf_matrix = tfidf.fit_transform(features)
    return tfidf_matrix

# calculate the cosine similarity
#@st.cache
def calculate_cosine_similarity(tfidf_matrix):
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim

# recommend the listings
def recommend_listings(cosine_sim, df2, id, top_n): # id is the id of the listing
    recommended_listings = [] # create an empty list to store the recommended listings
    idx = id - 1 # get the index of the listing
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False) # get the similarity scores in descending order
    top_n_indexes = list(score_series.iloc[1:top_n+1].index) # get the indexes of the top n similar listings
    for i in top_n_indexes: # append the names of the recommended listings to the list
        recommended_listings.append(list(listings['name'])[i]) # append the names of the recommended listings to the list
    return recommended_listings # return the list of recommended listings

In [103]:
# create the web app

# set the page config
st.set_page_config(page_title='Airbnb Rio Listings Recommender', page_icon=':house:', layout='wide', initial_sidebar_state='auto')
    
# set the title of the web app
st.title('Airbnb Rio Listings Recommender')

# load the data
def load_data():
    df2 = pd.read_csv('df2.csv')
    return df2

# show the 5 rows of the data
st.write(df.head())

# create the tf-idf matrix
tfidf_matrix = create_tfidf_matrix(df2)

# calculate the cosine similarity
cosine_sim = calculate_cosine_similarity(tfidf_matrix)

# create the sidebar
st.sidebar.header('User Input Features')
# create the user input features
id = st.sidebar.number_input('Enter the id of the listing', min_value=1, value=1)

# create the recommendation button
if st.sidebar.button('Recommend'):
    recommended_listings = recommend_listings(cosine_sim, df2, id, 10)
    st.header('Recommended Listings')
    for i in recommended_listings:
        st.write(i)

# write the name of the listing
st.header('Listing Name')

st.success(df2['name'][id-1])

# write the price of the listing

st.header('Listing Price')

st.success(df['price'][id-1])



# create a subheader
st.subheader('Listing Description')

# show the description of the listing
st.write(df2['description'][id-1])

# show the image of the listing
st.image(df2['picture_url'][id-1])

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [107]:
if id > len(df2):
    return ['Invalid ID']

SyntaxError: 'return' outside function (1814565325.py, line 2)

In [49]:
# stop_words_en = stopwords.words('english')
# stop_words_po = stopwords.words('portuguese')

# # Step 4: Define the features to be used for calculating similarity
# features = ['name', 'neighbourhood_cleansed', 'description', 'property_category', 'amenities']

# # Step 5: Create a function to calculate the similarity scores
# def get_similarity_matrix(df):
#     stop_words = stop_words_en + stop_words_po
#     tfidf = TfidfVectorizer(stop_words=stop_words)
#     feature_matrix = tfidf.fit_transform(df[features].fillna(''))
#     similarity_matrix = cosine_similarity(feature_matrix)
#     return similarity_matrix

# similarity_matrix = get_similarity_matrix(df)

# # Step 6: Create a function to generate recommendations
# def get_recommendations(df, similarity_matrix, listing_id, num_recommendations=10):
#     index = df[df['id'] == listing_id].index[0]
#     similarity_scores = list(enumerate(similarity_matrix[index]))
#     similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
#     similarity_scores = similarity_scores[1:num_recommendations+1]
#     indices = [i[0] for i in similarity_scores]
#     return df.iloc[indices]


In [57]:
# # Check if the listing ID is present in the DataFrame
# if listing_id in df['53733044'].values:
#     # Get the top recommended listings for the listing with the given ID
#     recommended_listings = get_recommendations(df, similarity_matrix, listing_id)
#     # Display the recommended listings
#     print(recommended_listings)
# else:
#     print(f'Listing ID {listing_id} not found in DataFrame.')

NameError: name 'listing_id' is not defined

In [51]:
# # Get the top recommended listings for a listing with ID 'your_listing_id'
# recommended_listings = get_recommendations(df, similarity_matrix, '53733044')
# recommended_listings

IndexError: index 0 is out of bounds for axis 0 with size 0

In [108]:
df2

,id,listing_url,name,description,picture_url,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,...,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,baths,property_category
0,783493769216852616,https://www.airbnb.com/rooms/783493769216852616,"Leme, Brasil",Apartamento aconchegante próximo à praia do Le...,https://a0.muscache.com/pictures/fe1f4b1b-a300...,100.0,67.0,0,2.0,2.0,...,NaN,NaN,0,2,2,0,0,NaN,1.0,Entire
1,703973293620197060,https://www.airbnb.com/rooms/703973293620197060,Suíte com entrada independente em casarão 1,"Casa ampla, em excelente localização, situada ...",https://a0.muscache.com/pictures/miso/Hosting-...,70.0,100.0,0,4.0,5.0,...,4.00,5.00,1,4,0,4,0,0.26,1.0,Private
2,782895997622988215,https://www.airbnb.com/rooms/782895997622988215,Apartamento próximo ao metrô,Sua família vai estar perto de tudo ao ficar n...,https://a0.muscache.com/pictures/d48a6798-d1c8...,100.0,50.0,0,1.0,1.0,...,NaN,NaN,0,1,1,0,0,NaN,1.0,Entire
3,23768085,https://www.airbnb.com/rooms/23768085,Vamos a praia,"Ótimo apartamento para duas pessoas, de frente...",https://a0.muscache.com/pictures/e43b686c-c4b4...,100.0,100.0,0,1.0,2.0,...,4.98,4.78,1,1,1,0,0,1.59,1.0,Entire
4,10463735,https://www.airbnb.com/rooms/10463735,Sobrado aconchegante e espaçoso,"Sobrado amplo, arejado, com arquitetura inusit...",https://a0.muscache.com/pictures/52ffaa78-2e94...,78.0,100.0,0,6.0,6.0,...,2.00,1.00,1,6,2,4,0,0.03,2.0,Entire
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28286,689013693819928481,https://www.airbnb.com/rooms/689013693819928481,Apartamento no Recreio para o Rock in Rio,ótimo apartamento no Recreio dos Bandeirantes....,https://a0.muscache.com/pictures/miso/Hosting-...,NaN,100.0,0,1.0,1.0,...,5.00,5.00,1,1,1,0,0,0.27,2.0,Entire
28287,23414904,https://www.airbnb.com/rooms/23414904,Nice tranquil apartment in Copacabana,"Nice cosy apartment , for 2 guests . 5 minutes...",https://a0.muscache.com/pictures/64ab8162-d9c8...,100.0,100.0,1,1.0,1.0,...,4.92,4.84,0,1,1,0,0,0.86,1.0,Entire
28288,48489080,https://www.airbnb.com/rooms/48489080,Praia de Botafogo ao lado do Metrô,NaN,https://a0.muscache.com/pictures/fc04ec31-88c3...,99.0,93.0,0,14.0,31.0,...,4.89,4.56,0,13,13,0,0,0.95,1.0,Entire
28289,43109156,https://www.airbnb.com/rooms/43109156,Lindo apart. 250 mega internet a 300m Praia Leme,"Vista verde, Silencioso, Porteiro 24h<br />Loc...",https://a0.muscache.com/pictures/a0199371-63a6...,100.0,96.0,1,10.0,10.0,...,4.89,4.63,1,4,4,0,0,0.60,1.0,Entire


In [109]:
df2.shape

(28291, 50)

In [110]:
df2

,id,listing_url,name,description,picture_url,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,...,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,baths,property_category
0,783493769216852616,https://www.airbnb.com/rooms/783493769216852616,"Leme, Brasil",Apartamento aconchegante próximo à praia do Le...,https://a0.muscache.com/pictures/fe1f4b1b-a300...,100.0,67.0,0,2.0,2.0,...,NaN,NaN,0,2,2,0,0,NaN,1.0,Entire
1,703973293620197060,https://www.airbnb.com/rooms/703973293620197060,Suíte com entrada independente em casarão 1,"Casa ampla, em excelente localização, situada ...",https://a0.muscache.com/pictures/miso/Hosting-...,70.0,100.0,0,4.0,5.0,...,4.00,5.00,1,4,0,4,0,0.26,1.0,Private
2,782895997622988215,https://www.airbnb.com/rooms/782895997622988215,Apartamento próximo ao metrô,Sua família vai estar perto de tudo ao ficar n...,https://a0.muscache.com/pictures/d48a6798-d1c8...,100.0,50.0,0,1.0,1.0,...,NaN,NaN,0,1,1,0,0,NaN,1.0,Entire
3,23768085,https://www.airbnb.com/rooms/23768085,Vamos a praia,"Ótimo apartamento para duas pessoas, de frente...",https://a0.muscache.com/pictures/e43b686c-c4b4...,100.0,100.0,0,1.0,2.0,...,4.98,4.78,1,1,1,0,0,1.59,1.0,Entire
4,10463735,https://www.airbnb.com/rooms/10463735,Sobrado aconchegante e espaçoso,"Sobrado amplo, arejado, com arquitetura inusit...",https://a0.muscache.com/pictures/52ffaa78-2e94...,78.0,100.0,0,6.0,6.0,...,2.00,1.00,1,6,2,4,0,0.03,2.0,Entire


In [111]:
features = df2[['name', 'neighbourhood_cleansed', 'description', 'property_category', 'amenities']]
features

,name,neighbourhood_cleansed,description,property_category,amenities
0,"Leme, Brasil",Leme,Apartamento aconchegante próximo à praia do Le...,Entire,"[""Wifi"", ""Dedicated workspace"", ""Kitchen"", ""Fi..."
1,Suíte com entrada independente em casarão 1,Botafogo,"Casa ampla, em excelente localização, situada ...",Private,"[""Wifi"", ""Lock on bedroom door"", ""TV"", ""Coffee..."
2,Apartamento próximo ao metrô,Laranjeiras,Sua família vai estar perto de tudo ao ficar n...,Entire,"[""Cooking basics"", ""Clothing storage"", ""Washer..."
3,Vamos a praia,Barra da Tijuca,"Ótimo apartamento para duas pessoas, de frente...",Entire,"[""Ping pong table"", ""Private patio or balcony""..."
4,Sobrado aconchegante e espaçoso,Laranjeiras,"Sobrado amplo, arejado, com arquitetura inusit...",Entire,"[""Iron"", ""TV with standard cable"", ""Wifi"", ""Ki..."
...,...,...,...,...,...
28286,Apartamento no Recreio para o Rock in Rio,Recreio dos Bandeirantes,ótimo apartamento no Recreio dos Bandeirantes....,Entire,"[""Kitchen"", ""Fire extinguisher"", ""Air conditio..."
28287,Nice tranquil apartment in Copacabana,Copacabana,"Nice cosy apartment , for 2 guests . 5 minutes...",Entire,"[""Self check-in"", ""Private patio or balcony"", ..."
28288,Praia de Botafogo ao lado do Metrô,Botafogo,NaN,Entire,"[""Self check-in"", ""Cooking basics"", ""Carbon mo..."
28289,Lindo apart. 250 mega internet a 300m Praia Leme,Botafogo,"Vista verde, Silencioso, Porteiro 24h<br />Loc...",Entire,"[""Window AC unit"", ""Coffee maker: pour-over co..."
